In [ ]:
import xarray as xr
import numpy as np
import scipy.stats as stats
from scipy.ndimage import gaussian_filter, uniform_filter

from scores.probability import (
    crps_for_ensemble,
    tail_twcrps_for_ensemble,
    crps_for_ensemble,
)
from scores.continuous import mse

from scores.processing import broadcast_and_match_nan
import matplotlib.pyplot as plt


np.random.seed(100)

In [ ]:
X_LEN = 100
Y_LEN = 100

NEIGHBORHOOD = 7

NOISY_CONVECTION_VMAX = 30
BLOB_CONVECTION_VMAX = 40
ACCUM_CONVECTION_VMAX = 200

TW_QUANTILE = 0.8


def create_neighourhood_ensemble(fcst, neighborhood, remove_incomplete_ens=True):
    ensemble = fcst.rolling(
        dim=dict(x=neighborhood, y=neighborhood), center=True
    ).construct(x="i", y="j")
    ensemble = ensemble.stack(ens_mem=("i", "j"))
    if remove_incomplete_ens:
        ensemble = ensemble.where(~np.isnan(ensemble).any(dim="ens_mem"), np.nan)
    # Clean up coordinates for crps calculation
    ensemble = ensemble.drop_vars("ens_mem")
    ensemble = ensemble.assign_coords(ens_mem=np.arange(neighborhood**2))
    return ensemble

In [ ]:


# Set the value of a
a = 5

# Generate random values from a skew normal distribution
random_values = stats.skewnorm.rvs(a=a, scale=10, size=1000)

# Plot the histogram of the random values
plt.hist(random_values, bins=30, density=True, alpha=0.7)

# Plot the probability density function (PDF) of the skew normal distribution
x = np.linspace(
    stats.skewnorm.ppf(0.001, a=a, scale=10),
    stats.skewnorm.ppf(0.999, a=a, scale=10),
    100,
)
plt.plot(x, stats.skewnorm.pdf(x, a=a, scale=10), "r-", lw=2)

# Set the plot title and labels
plt.title(f"Skewnorm Distribution (a={a}, scale=10)")
plt.xlabel("Value")
plt.ylabel("Density")

# Show the plot
plt.show()

# Noisy convection

In [ ]:
# Generate random values from a skew normal distribution
obs_convection = stats.skewnorm.rvs(a=10, scale=10, size=(Y_LEN, X_LEN)).clip(min=0)
obs_convection = xr.DataArray(
    obs_convection,
    dims=("y", "x"),
    coords={"x": np.arange(X_LEN), "y": np.arange(Y_LEN)},
)

fcst_convection = stats.skewnorm.rvs(a=10, scale=10, size=(Y_LEN, X_LEN)).clip(min=0)
fcst_convection = xr.DataArray(
    fcst_convection,
    dims=("y", "x"),
    coords={"x": np.arange(X_LEN), "y": np.arange(Y_LEN)},
)

fcst_clim = data = np.full((Y_LEN, X_LEN), obs_convection.mean().values)
fcst_clim = xr.DataArray(
    fcst_clim, dims=("y", "x"), coords={"x": np.arange(X_LEN), "y": np.arange(Y_LEN)}
)
fcst_uniform_filter = xr.apply_ufunc(
    uniform_filter, fcst_convection, kwargs=dict(size=NEIGHBORHOOD)
)
fcst_gaussian_filter = xr.apply_ufunc(
    gaussian_filter, fcst_convection, kwargs=dict(sigma=1)
)

# Visualise data (before ensemble creation)
fig, axs = plt.subplots(2, 2, figsize=(10, 10))

# Plot obs_convection
im1 = axs[0, 0].imshow(obs_convection, vmin=0, vmax=NOISY_CONVECTION_VMAX)
axs[0, 0].set_title("Obs noisy convection")

# Plot fcst_convection
im2 = axs[0, 1].imshow(fcst_convection, vmin=0, vmax=NOISY_CONVECTION_VMAX)
axs[0, 1].set_title("Forecast noisy convection")

# Plot fcst neighbourhood mean
im3 = axs[1, 0].imshow(fcst_uniform_filter, vmin=0, vmax=NOISY_CONVECTION_VMAX)
axs[1, 0].set_title("Forecast uniform filter")

# Plot fcst gaussian filter
im4 = axs[1, 1].imshow(fcst_gaussian_filter, vmin=0, vmax=NOISY_CONVECTION_VMAX)
axs[1, 1].set_title("Forecast gaussian filter")

# Add colorbars
fig.colorbar(im1, ax=axs[0, 0], label="mm")
fig.colorbar(im2, ax=axs[0, 1], label="mm")
fig.colorbar(im3, ax=axs[1, 0], label="mm")
fig.colorbar(im4, ax=axs[1, 1], label="mm")

plt.show()

In [ ]:
# Create ensembles
fcst_conv_ens = create_neighourhood_ensemble(fcst_convection, NEIGHBORHOOD)
fcst_clim_ens = create_neighourhood_ensemble(fcst_clim, NEIGHBORHOOD)
fcst_uniform_filter_ens = create_neighourhood_ensemble(
    fcst_uniform_filter, NEIGHBORHOOD
)
fcst_gaussian_filter_ens = create_neighourhood_ensemble(
    fcst_gaussian_filter, NEIGHBORHOOD
)

In [ ]:
crps_noisy_conv = crps_for_ensemble(
    fcst_conv_ens, obs_convection, ensemble_member_dim="ens_mem"
).values
crps_clim_mean = crps_for_ensemble(
    fcst_clim_ens, obs_convection, ensemble_member_dim="ens_mem"
).values
crps_uniform_filter = crps_for_ensemble(
    fcst_uniform_filter_ens, obs_convection, ensemble_member_dim="ens_mem"
).values
crps_gaussian_filter = crps_for_ensemble(
    fcst_gaussian_filter_ens, obs_convection, ensemble_member_dim="ens_mem"
).values

print(f"CRPS for noisy convection: {crps_noisy_conv}")
print(f"CRPS for climatology: {crps_clim_mean}")
print(f"CRPS for uniform mean: {crps_uniform_filter}")
print(f"CRPS for gaussian mean: {crps_gaussian_filter}")

twcrps_noisy_conv = tail_twcrps_for_ensemble(
    fcst_conv_ens,
    obs_convection,
    ensemble_member_dim="ens_mem",
    tail="upper",
    threshold=np.quantile(obs_convection, TW_QUANTILE),
).values
twcrps_clim_mean = tail_twcrps_for_ensemble(
    fcst_clim_ens,
    obs_convection,
    ensemble_member_dim="ens_mem",
    tail="upper",
    threshold=np.quantile(obs_convection, TW_QUANTILE),
).values
twcrps_uniform_filter = tail_twcrps_for_ensemble(
    fcst_uniform_filter_ens,
    obs_convection,
    ensemble_member_dim="ens_mem",
    tail="upper",
    threshold=np.quantile(obs_convection, TW_QUANTILE),
).values
twcrps_gaussian_filter = tail_twcrps_for_ensemble(
    fcst_gaussian_filter_ens,
    obs_convection,
    ensemble_member_dim="ens_mem",
    tail="upper",
    threshold=np.quantile(obs_convection, TW_QUANTILE),
).values

print(f"TWCRPS for noisy convection: {twcrps_noisy_conv}")
print(f"TWCRPS for climatology: {twcrps_clim_mean}")
print(f"TWCRPS for uniform mean: {twcrps_uniform_filter}")
print(f"TWCRPS for gaussian mean: {twcrps_gaussian_filter}")


mse_noisy_conv = mse(fcst_convection, obs_convection).values
mse_clim_mean = mse(fcst_clim, obs_convection).values
mse_uniform_filter = mse(fcst_uniform_filter, obs_convection).values
mse_gaussian_filter = mse(fcst_gaussian_filter, obs_convection).values
print(f"MSE for noisy convection: {mse_noisy_conv}")
print(f"MSE for climatology: {mse_clim_mean}")
print(f"MSE for uniform filter: {mse_uniform_filter}")
print(f"MSE for gaussian filter: {mse_gaussian_filter}")

# Blobby convection that is stronger in the east

In [ ]:
# Generate blobby convection observations
random_data = stats.skewnorm.rvs(a=0, scale=4, size=(Y_LEN, X_LEN)) * 10
random_data = random_data * np.linspace(0.25, 2, Y_LEN)
smoothed_data = gaussian_filter(random_data, sigma=1).clip(min=0)
obs_convection_blobs = xr.DataArray(smoothed_data, dims=["y", "x"])

# Generate perfect forecast that is offset by 5 in x and -5 in y
fcst = obs_convection_blobs.copy()
fcst_shifted = fcst.shift(x=5, y=-5)
fcst_shifted

# Generate blobby convection forecasts
random_data = stats.skewnorm.rvs(a=0, scale=4, size=(Y_LEN, X_LEN)) * 10
random_data = random_data * np.linspace(0.25, 2, Y_LEN)
smoothed_data = gaussian_filter(random_data, sigma=1).clip(min=0)
fcst_convection_blobs = xr.DataArray(smoothed_data, dims=["y", "x"])


fcst_blobby_uniform_filter = xr.apply_ufunc(
    uniform_filter, fcst_convection_blobs, kwargs=dict(size=NEIGHBORHOOD)
)
fcst_blobby_gaussian_filter = xr.apply_ufunc(
    gaussian_filter, fcst_convection_blobs, kwargs=dict(sigma=1)
)

(
    obs_convection_blobs,
    fcst_shifted,
    fcst_convection_blobs,
    fcst_blobby_uniform_filter,
    fcst_blobby_gaussian_filter,
) = broadcast_and_match_nan(
    obs_convection_blobs,
    fcst_shifted,
    fcst_convection_blobs,
    fcst_blobby_uniform_filter,
    fcst_blobby_gaussian_filter,
)

fig, axs = plt.subplots(2, 3, figsize=(15, 10))

# Plot obs_convection
im1 = axs[0, 0].imshow(obs_convection_blobs, vmin=0, vmax=BLOB_CONVECTION_VMAX)
axs[0, 0].set_title("Obs blobby convection")

# Plot fcst_convection
im2 = axs[0, 1].imshow(fcst_convection_blobs, vmin=0, vmax=BLOB_CONVECTION_VMAX)
axs[0, 1].set_title("Forecast blobby convection")

# Add colorbars
fig.colorbar(im1, ax=axs[0, 0], label="mm")
fig.colorbar(im2, ax=axs[0, 1], label="mm")

# Plot fcst_shifted
im3 = axs[1, 0].imshow(fcst_shifted, vmin=0, vmax=BLOB_CONVECTION_VMAX)
axs[1, 0].set_title("Perfect forecast shifted")

# Add colorbar
fig.colorbar(im3, ax=axs[1, 0], label="mm")

# Plot fcst_blobby_uniform_filter
im4 = axs[1, 1].imshow(fcst_blobby_uniform_filter, vmin=0, vmax=BLOB_CONVECTION_VMAX)
axs[1, 1].set_title("Forecast blobby uniform filter")

# Add colorbar
fig.colorbar(im4, ax=axs[1, 1], label="mm")

# Plot fcst_blobby_gaussian_filter
im5 = axs[0, 2].imshow(fcst_blobby_gaussian_filter, vmin=0, vmax=BLOB_CONVECTION_VMAX)
axs[0, 2].set_title("Forecast blobby gaussian filter")

# Add colorbar
fig.colorbar(im5, ax=axs[0, 2], label="mm")
# Hide the missing subplot
axs[1, 2].axis("off")
plt.show()

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(15, 10))

# Plot obs_convection
im1 = axs[0, 0].imshow(obs_convection_blobs, vmin=10, vmax=BLOB_CONVECTION_VMAX)
axs[0, 0].set_title("Obs blobby convection")

# Plot fcst_convection
im2 = axs[0, 1].imshow(fcst_convection_blobs, vmin=10, vmax=BLOB_CONVECTION_VMAX)
axs[0, 1].set_title("Forecast blobby convection")

# Add colorbars
fig.colorbar(im1, ax=axs[0, 0], label="mm")
fig.colorbar(im2, ax=axs[0, 1], label="mm")

# Plot fcst_shifted
im3 = axs[1, 0].imshow(fcst_shifted, vmin=10, vmax=BLOB_CONVECTION_VMAX)
axs[1, 0].set_title("Perfect forecast shifted")

# Add colorbar
fig.colorbar(im3, ax=axs[1, 0], label="mm")

# Plot fcst_blobby_uniform_filter
im4 = axs[1, 1].imshow(fcst_blobby_uniform_filter, vmin=10, vmax=BLOB_CONVECTION_VMAX)
axs[1, 1].set_title("Forecast blobby uniform filter")

# Add colorbar
fig.colorbar(im4, ax=axs[1, 1], label="mm")

# Plot fcst_blobby_gaussian_filter
im5 = axs[0, 2].imshow(fcst_blobby_gaussian_filter, vmin=10, vmax=BLOB_CONVECTION_VMAX)
axs[0, 2].set_title("Forecast blobby gaussian filter")

# Add colorbar
fig.colorbar(im5, ax=axs[0, 2], label="mm")
# Hide the missing subplot
axs[1, 2].axis("off")
plt.show()

In [ ]:
# Create ensembles
fcst_blobs_ens = create_neighourhood_ensemble(fcst_convection_blobs, NEIGHBORHOOD)
fcst_shifted_ens = create_neighourhood_ensemble(fcst_shifted, NEIGHBORHOOD)
fcst_blobby_uniform_filter_ens = create_neighourhood_ensemble(
    fcst_blobby_uniform_filter, NEIGHBORHOOD
)
fcst_blobby_gaussian_filter_ens = create_neighourhood_ensemble(
    fcst_blobby_gaussian_filter, NEIGHBORHOOD
)

In [ ]:
# Calculate CRPS
crps_blobs = crps_for_ensemble(
    fcst_blobs_ens, obs_convection_blobs, ensemble_member_dim="ens_mem"
).values
crps_shifted = crps_for_ensemble(
    fcst_shifted_ens, obs_convection_blobs, ensemble_member_dim="ens_mem"
).values
crps_blobby_uniform_filter = crps_for_ensemble(
    fcst_blobby_uniform_filter_ens, obs_convection_blobs, ensemble_member_dim="ens_mem"
).values
crps_blobby_gaussian_filter = crps_for_ensemble(
    fcst_blobby_gaussian_filter_ens, obs_convection_blobs, ensemble_member_dim="ens_mem"
).values
# Calculate TWCRPS
twcrps_blobs = tail_twcrps_for_ensemble(
    fcst_blobs_ens,
    obs_convection_blobs,
    ensemble_member_dim="ens_mem",
    tail="upper",
    threshold=obs_convection_blobs.quantile(TW_QUANTILE).item(),
).values
twcrps_shifted = tail_twcrps_for_ensemble(
    fcst_shifted_ens,
    obs_convection_blobs,
    ensemble_member_dim="ens_mem",
    tail="upper",
    threshold=obs_convection_blobs.quantile(TW_QUANTILE).item(),
).values
twcrps_blobby_uniform_filter = tail_twcrps_for_ensemble(
    fcst_blobby_uniform_filter_ens,
    obs_convection_blobs,
    ensemble_member_dim="ens_mem",
    tail="upper",
    threshold=obs_convection_blobs.quantile(TW_QUANTILE).item(),
).values
twcrps_blobby_gaussian_filter = tail_twcrps_for_ensemble(
    fcst_blobby_gaussian_filter_ens,
    obs_convection_blobs,
    ensemble_member_dim="ens_mem",
    tail="upper",
    threshold=obs_convection_blobs.quantile(TW_QUANTILE).item(),
).values

# Calculate MSE
mse_blobs = mse(fcst_convection_blobs, obs_convection_blobs).values
mse_shifted = mse(fcst_shifted, obs_convection_blobs).values
mse_blobby_uniform_filter = mse(fcst_blobby_uniform_filter, obs_convection_blobs).values
mse_blobby_gaussian_filter = mse(
    fcst_blobby_gaussian_filter, obs_convection_blobs
).values
print(f"CRPS for blobs: {crps_blobs}")
print(f"CRPS for shifted: {crps_shifted}")
print(f"CRPS for blobby uniform filter: {crps_blobby_uniform_filter}")
print(f"CRPS for blobby gaussian filter: {crps_blobby_gaussian_filter}")
print(f"TWCRPS for blobs: {twcrps_blobs}")
print(f"TWCRPS for shifted: {twcrps_shifted}")
print(f"TWCRPS for blobby uniform filter: {twcrps_blobby_uniform_filter}")
print(f"TWCRPS for blobby gaussian filter: {twcrps_blobby_gaussian_filter}")
print(f"MSE for blobs: {mse_blobs}")
print(f"MSE for shifted: {mse_shifted}")
print(f"MSE for blobby uniform filter: {mse_blobby_uniform_filter}")
print(f"MSE for blobby gaussian filter: {mse_blobby_gaussian_filter}")

# Now create patterns that look more like multi-hour accumulated precip fields

In [ ]:
VIM_BLOB = 50


def create_accum_like_obj():
    # Create coordinate arrays
    x = np.linspace(-1, 1, X_LEN)
    y = np.linspace(-1, 1, Y_LEN)
    xv, yv = np.meshgrid(x, y)
    # Define the oval equation with increasing intensity towards the center
    oval = np.exp(-((yv**2) / 0.6**2 + (xv**2) / 0.3**2)) * 450
    # Generate random noise
    noise = np.random.normal(0, 1, size=(X_LEN, Y_LEN))
    # Apply Gaussian filter to create blobby effect
    blobby_noise = gaussian_filter(noise, sigma=3)
    blobby_noise = blobby_noise - np.min(blobby_noise)
    accum = oval * blobby_noise
    da = xr.DataArray(
        accum, dims=["y", "x"], coords={"y": np.arange(Y_LEN), "x": np.arange(X_LEN)}
    )
    return da


# Create the DataArray
obs = create_accum_like_obj()

# Generate perfect forecast that is offset by 5 in x and -5 in y
fcst = obs.copy()
fcst_shifted = fcst.shift(x=5, y=-5)

# Create a fcst that is the observations transposed
fcst_transposed = fcst.transpose("x", "y")
fcst_transposed = fcst_transposed.rename({"x": "y", "y": "x"})

# Create fcst from the same distirbution
fcst = create_accum_like_obj()

fcst_uniform_filter = xr.apply_ufunc(
    uniform_filter, fcst, kwargs=dict(size=NEIGHBORHOOD * 2)
)
fcst_gaussian_filter = xr.apply_ufunc(gaussian_filter, fcst, kwargs=dict(sigma=3))
obs, fcst, fcst_shifted, fcst_transposed, fcst_uniform_filter, fcst_gaussian_filter = (
    broadcast_and_match_nan(
        obs,
        fcst,
        fcst_shifted,
        fcst_transposed,
        fcst_uniform_filter,
        fcst_gaussian_filter,
    )
)

# clim fcst
fcst_clim = data = np.full((Y_LEN, X_LEN), obs.mean().values)
fcst_clim = xr.DataArray(
    fcst_clim, dims=("y", "x"), coords={"x": np.arange(X_LEN), "y": np.arange(Y_LEN)}
)

fig, axs = plt.subplots(2, 3, figsize=(15, 10))

# Plot obs
im1 = axs[0, 0].imshow(obs, vmin=VIM_BLOB, vmax=ACCUM_CONVECTION_VMAX)
axs[0, 0].set_title("Obs")

# Plot fcst
im2 = axs[0, 1].imshow(fcst, vmin=VIM_BLOB, vmax=ACCUM_CONVECTION_VMAX)
axs[0, 1].set_title("Forecast")

# Plot fcst_shifted
im3 = axs[0, 2].imshow(fcst_shifted, vmin=VIM_BLOB, vmax=ACCUM_CONVECTION_VMAX)
axs[0, 2].set_title("Forecast Shifted")

# Plot fcst_transposed
im4 = axs[1, 0].imshow(fcst_transposed, vmin=VIM_BLOB, vmax=ACCUM_CONVECTION_VMAX)
axs[1, 0].set_title("Forecast Transposed")

# Plot fcst_uniform_filter
im5 = axs[1, 1].imshow(fcst_uniform_filter, vmin=VIM_BLOB, vmax=ACCUM_CONVECTION_VMAX)
axs[1, 1].set_title("Forecast Uniform Filter")

# Plot fcst_gaussian_filter
im6 = axs[1, 2].imshow(fcst_gaussian_filter, vmin=VIM_BLOB, vmax=ACCUM_CONVECTION_VMAX)
axs[1, 2].set_title("Forecast Gaussian Filter")

# Add colorbars
fig.colorbar(im1, ax=axs[0, 0], label="mm")
fig.colorbar(im2, ax=axs[0, 1], label="mm")
fig.colorbar(im3, ax=axs[0, 2], label="mm")
fig.colorbar(im4, ax=axs[1, 0], label="mm")
fig.colorbar(im5, ax=axs[1, 1], label="mm")
fig.colorbar(im6, ax=axs[1, 2], label="mm")

plt.show()

In [ ]:
ACCUM_NEIGHBORHOOD = 21

# Create ensembles
fcst_ens = create_neighourhood_ensemble(fcst, ACCUM_NEIGHBORHOOD)
fcst_shifted_ens = create_neighourhood_ensemble(fcst_shifted, ACCUM_NEIGHBORHOOD)
fcst_transposed_ens = create_neighourhood_ensemble(fcst_transposed, ACCUM_NEIGHBORHOOD)
fcst_uniform_filter_ens = create_neighourhood_ensemble(
    fcst_uniform_filter, ACCUM_NEIGHBORHOOD
)
fcst_gaussian_filter_ens = create_neighourhood_ensemble(
    fcst_gaussian_filter, ACCUM_NEIGHBORHOOD
)
fcst_clim_ens = create_neighourhood_ensemble(fcst_clim, ACCUM_NEIGHBORHOOD)

In [ ]:
# Calculate CRPS
crps = crps_for_ensemble(fcst_ens, obs, ensemble_member_dim="ens_mem").values
print(f"CRPS for fcst from the same distribution: {crps}")
crps_shifted = crps_for_ensemble(
    fcst_shifted_ens, obs, ensemble_member_dim="ens_mem"
).values
print(f"CRPS for fcst shifted: {crps_shifted}")
crps_transposed = crps_for_ensemble(
    fcst_transposed_ens, obs, ensemble_member_dim="ens_mem"
).values
print(f"CRPS for fcst transposed: {crps_transposed}")
crps_uniform_filter = crps_for_ensemble(
    fcst_uniform_filter_ens, obs, ensemble_member_dim="ens_mem"
).values
print(f"CRPS for fcst uniform filter: {crps_uniform_filter}")
crps_gaussian_filter = crps_for_ensemble(
    fcst_gaussian_filter_ens, obs, ensemble_member_dim="ens_mem"
).values
print(f"CRPS for fcst gaussian filter: {crps_gaussian_filter}")
crps_clim = crps_for_ensemble(fcst_clim_ens, obs, ensemble_member_dim="ens_mem").values
print(f"CRPS for fcst climatology: {crps_clim}")

# Calculate TWCRPS
twcrps = tail_twcrps_for_ensemble(
    fcst_ens, obs, ensemble_member_dim="ens_mem", tail="upper", threshold=50
).values
print(f"TWCRPS for fcst from the same distribution: {twcrps}")
twcrps_shifted = tail_twcrps_for_ensemble(
    fcst_shifted_ens, obs, ensemble_member_dim="ens_mem", tail="upper", threshold=50
).values
print(f"TWCRPS for fcst shifted: {twcrps_shifted}")
twcrps_transposed = tail_twcrps_for_ensemble(
    fcst_transposed_ens, obs, ensemble_member_dim="ens_mem", tail="upper", threshold=50
).values
print(f"TWCRPS for fcst transposed: {twcrps_transposed}")
twcrps_uniform_filter = tail_twcrps_for_ensemble(
    fcst_uniform_filter_ens,
    obs,
    ensemble_member_dim="ens_mem",
    tail="upper",
    threshold=50,
).values
print(f"TWCRPS for fcst uniform filter: {twcrps_uniform_filter}")
twcrps_gaussian_filter = tail_twcrps_for_ensemble(
    fcst_gaussian_filter_ens,
    obs,
    ensemble_member_dim="ens_mem",
    tail="upper",
    threshold=50,
).values
print(f"TWCRPS for fcst gaussian filter: {twcrps_gaussian_filter}")
twcrps_clim = tail_twcrps_for_ensemble(
    fcst_clim_ens, obs, ensemble_member_dim="ens_mem", tail="upper", threshold=50
).values
print(f"TWCRPS for fcst climatology: {twcrps_clim}")

# # Calculate MSE
mse_fcst = mse(fcst, obs).values
mse_shifted = mse(fcst_shifted, obs).values
mse_transposed = mse(fcst_transposed, obs).values
mse_uniform_filter = mse(fcst_uniform_filter, obs).values
mse_gaussian_filter = mse(fcst_gaussian_filter, obs).values
mse_clim = mse(fcst_clim, obs).values
print(f"MSE for fcst from the same distribution: {mse_fcst}")
print(f"MSE for fcst shifted: {mse_shifted}")
print(f"MSE for fcst transposed: {mse_transposed}")
print(f"MSE for fcst uniform filter: {mse_uniform_filter}")
print(f"MSE for fcst gaussian filter: {mse_gaussian_filter}")
print(f"MSE for fcst climatology: {mse_clim}")

# performance testing

In [ ]:
def ensemble_to_cdf(ensemble_da, ensemble_mem_dim="ens_mem"):
    # Step 1: Sort the ensemble along the ensemble dimension, keeping other dimensions intact
    sorted_ensemble = ensemble_da.sortby(ensemble_mem_dim)

    # Step 2: Calculate the rank array that represents the CDF values
    ensemble_size = sorted_ensemble.sizes[ensemble_mem_dim]
    cdf_values = np.arange(1, ensemble_size + 1) / ensemble_size

    # Step 3: Expand the rank array to match the dimensions of sorted_ensemble
    cdf_da = xr.DataArray(cdf_values, dims=[ensemble_mem_dim])
    cdf_expanded = cdf_da.broadcast_like(sorted_ensemble)

    return cdf_expanded


fcst_cdf = ensemble_to_cdf(fcst_ens)

In [ ]:
# Step 1: Sort the ensemble members
sorted_ens = fcst_ens.copy()
axes_num = sorted_ens.get_axis_num("ens_mem")
sorted_ens.values = np.sort(fcst_ens.values, axes_num)

ensemble_size = sorted_ens.sizes["ens_mem"]
n = np.arange(1, ensemble_size + 1) / ensemble_size
cumulative_probs = xr.DataArray(
    np.arange(1, n + 1) / n,
    dims=["ens_member"],
    coords={"ens_member": sorted_ens["ens_member"]},
)


# Step 3: Expand the rank array to match the dimensions of sorted_ens
cdf_da = xr.DataArray(cdf_values, dims=["ens_mem"])
# cdf_expanded = cdf_da.broadcast_like(sorted_ens)

# # Step 4: Assign the expanded rank array as the CDF values
# sorted_ens["cdf"] = cdf_expanded

# # Print the sorted ensemble with CDF values
# print(sorted_ens)
ensemble_size

In [ ]:
sorted_ens = fcst_ens.copy()
axes_num = sorted_ens.get_axis_num("ens_mem")
sorted_ens.values = np.sort(fcst_ens.values, axes_num)
sorted_ens.sel(x=50, y=50).plot()

In [ ]:
sorted_ens.size

In [ ]:
fcst_ens_sorted = fcst_ens.sortby("ens_mem")
fcst_ens_sorted.sel(x=50, y=50).plot()

In [ ]:
data = xr.DataArray(
    np.random.rand(5, 4, 3),  # Random data with dimensions a, b, c
    dims=["a", "b", "c"],
    coords={
        "a": [3, 1, 4, 2, 5],  # Coordinate values for dimension 'a'
        "b": np.arange(4),
        "c": np.arange(3),
    },
)

sorted_data = data.copy()
axes_num = sorted_data.get_axis_num("a")
sorted_data.values = np.sort(data.values, axes_num)

ensemble_size = sorted_data.sizes["a"]
# sorted_data
cdf_values = np.arange(1, ensemble_size + 1) / ensemble_size
cdf_da = xr.DataArray(cdf_values, dims=["a"])
cdf_expanded = cdf_da.broadcast_like(sorted_data)
sorted_data["cdf"] = cdf_expanded

sorted_data

In [ ]:
sorted_data = data.copy()
axes_num = sorted_data.get_axis_num("a")
sorted_data.values = np.sort(data.values, axes_num)

In [ ]:
sorted_data.sel(b=1, c=1)

In [ ]:
sorted_data.get_axis_num("a")

In [ ]:
fcst1 = [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
fcst1 = xr.DataArray(fcst1, dims=["ens_mem"], coords={"ens_mem": np.arange(len(fcst1))})

fcst2 = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
fcst2 = xr.DataArray(fcst2, dims=["ens_mem"], coords={"ens_mem": np.arange(len(fcst2))})

fcst2b = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
fcst2b = xr.DataArray(
    fcst2b, dims=["ens_mem"], coords={"ens_mem": np.arange(len(fcst2b))}
)

fcst3 = [0.1]
fcst3 = xr.DataArray(fcst3, dims=["ens_mem"], coords={"ens_mem": np.arange(len(fcst3))})

obs = xr.DataArray(1)

print(crps_for_ensemble(fcst1, obs, ensemble_member_dim="ens_mem"))
print(crps_for_ensemble(fcst3, obs, ensemble_member_dim="ens_mem"))

In [ ]:
print(crps_for_ensemble(fcst2, obs, ensemble_member_dim="ens_mem"))
print(crps_for_ensemble(fcst2b, obs, ensemble_member_dim="ens_mem"))

In [ ]:
exp_1 = (9 * 0.01 + 0.81) / 10
exp_3 = (9 * 0.1 + 0.9) / 10
print(exp_1, exp_3)

# No spatial dependence

In [ ]:
fcst_ens_0 = [0, 0, 0, 0, 0, 0, 0, 0, 1]
fcst_ens = xr.DataArray(
    fcst_ens_0, dims=["ens_mem"], coords={"ens_mem": np.arange(len(fcst_ens_0))}
)
fcst_det = xr.DataArray([1 / 9], dims=["ens_mem"], coords={"ens_mem": [1]})


crps_ensemble_list = []
crps_deterministic_list = []
for i in np.arange(1000):
    value = np.random.choice([0, 1], p=[1 / 9, 8 / 9])
    obs = xr.DataArray(obs)
    crps_ensemble_list.append(
        crps_for_ensemble(fcst_ens, obs, ensemble_member_dim="ens_mem").item()
    )
    # Generate deterministic forecast
    value2 = np.random.choice([0, 1], p=[8 / 9, 1 / 9])
    fcst_det = xr.DataArray([value2], dims=["ens_mem"], coords={"ens_mem": [1]})
    crps_deterministic_list.append(
        crps_for_ensemble(fcst_det, obs, ensemble_member_dim="ens_mem").item()
    )

print(np.mean(crps_ensemble_list), np.mean(crps_deterministic_list))

# Complete spatial dependence

In [ ]:
fcst_ens_0 = [0] * 9
fcst_ens_0 = xr.DataArray(
    fcst_ens_0, dims=["ens_mem"], coords={"ens_mem": np.arange(len(fcst_ens_0))}
)

fcst_ens_1 = [1] * 9
fcst_ens_1 = xr.DataArray(
    fcst_ens_1, dims=["ens_mem"], coords={"ens_mem": np.arange(len(fcst_ens_1))}
)
fcst_det = xr.DataArray([1 / 9], dims=["ens_mem"], coords={"ens_mem": [1]})


crps_ensemble_list = []
crps_deterministic_list = []
for i in np.arange(10000):
    value = np.random.choice([0, 1], p=[1 / 9, 8 / 9])
    obs = xr.DataArray(obs)
    # Generate ensemble with complete spatial dependence
    if np.random.random() < 8 / 9:
        fcst = fcst2
    else:
        fcst = fcst2b
    crps_ensemble_list.append(
        crps_for_ensemble(fcst, obs, ensemble_member_dim="ens_mem").item()
    )
    # Generate deterministic forecast
    value2 = np.random.choice([0, 1], p=[8 / 9, 1 / 9])

    fcst_det = xr.DataArray([value2], dims=["ens_mem"], coords={"ens_mem": [1]})
    crps_deterministic_list.append(
        crps_for_ensemble(fcst_det, obs, ensemble_member_dim="ens_mem").item()
    )

print(np.mean(crps_ensemble_list), np.mean(crps_deterministic_list))